In [15]:
import pandas as pd
import numpy as np

In [16]:
flights = pd.read_csv('flights.csv')
meta = pd.read_csv('airports_meta.csv', header=None, names = ["row_num", "airport_full_name", "airport_city", "country", "airport_code", 
                                                              "airport_random_id", "lat", "lon", "r_1", "r_2", "r_3", "timezone", "r_4", "r_5" ])

In [17]:
flights.head()

,CARRIER_CODE,ORIGIN,ORIGIN_STATE_ABR,DEST,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,...,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CARRIER_NAME,OriginLat,OriginLon,DestLat,DestLon
0,AA,CLT,NC,SYR,NY,5,2358.0,-7.0,146,131.0,...,0.0,0.0,0.0,0.0,0.0,American,35.214001,-80.943100,43.111198,-76.106300
1,DL,LAS,NV,ATL,GA,5,4.0,-1.0,645,637.0,...,0.0,0.0,0.0,0.0,0.0,Delta,36.080101,-115.152000,33.636700,-84.428101
2,AA,SEA,WA,CLT,NC,5,6.0,1.0,804,741.0,...,0.0,0.0,0.0,0.0,0.0,American,47.449001,-122.308998,35.214001,-80.943100
3,AA,SEA,WA,ORD,IL,10,11.0,1.0,605,547.0,...,0.0,0.0,0.0,0.0,0.0,American,47.449001,-122.308998,41.978600,-87.904800
4,AA,CLT,NC,JAX,FL,10,6.0,-4.0,125,123.0,...,0.0,0.0,0.0,0.0,0.0,American,35.214001,-80.943100,30.494101,-81.687897


In [18]:
flights.shape

(9778, 25)

In [19]:
flights.columns

Index([u'CARRIER_CODE', u'ORIGIN', u'ORIGIN_STATE_ABR', u'DEST',
       u'DEST_STATE_ABR', u'CRS_DEP_TIME', u'DEP_TIME', u'DEP_DELAY',
       u'CRS_ARR_TIME', u'ARR_TIME', u'ARR_DELAY', u'CANCELLED',
       u'CANCELLATION_CODE', u'ACTUAL_ELAPSED_TIME', u'DISTANCE',
       u'CARRIER_DELAY', u'WEATHER_DELAY', u'NAS_DELAY', u'SECURITY_DELAY',
       u'LATE_AIRCRAFT_DELAY', u'CARRIER_NAME', u'OriginLat', u'OriginLon',
       u'DestLat', u'DestLon'],
      dtype='object')

In [20]:
outbound_flights = pd.DataFrame(flights.groupby(['ORIGIN'], as_index=False).size().rename('outbound'))
inbound_flights = pd.DataFrame(flights.groupby(['DEST'], as_index=False).size().rename('inbound'))

In [21]:
merged_df = outbound_flights.merge(inbound_flights, left_index=True, right_index=True)

In [22]:
merged_df['total'] = merged_df['inbound'] + merged_df['outbound']

In [23]:
merged_df.head()

,outbound,inbound,total
ABQ,31,30,61
ACY,11,11,22
ADQ,1,1,2
ALB,18,19,37
AMA,4,4,8


In [24]:
top_busiest = list(merged_df.sort_values('total', ascending=False)[:30].index)

In [25]:
top_busiest

['ATL',
 'LAX',
 'DEN',
 'PHX',
 'MCO',
 'LAS',
 'DFW',
 'ORD',
 'SFO',
 'SEA',
 'JFK',
 'FLL',
 'CLT',
 'BOS',
 'MIA',
 'BWI',
 'EWR',
 'MDW',
 'IAH',
 'MSP',
 'TPA',
 'SAN',
 'LGA',
 'DTW',
 'PHL',
 'SLC',
 'DAL',
 'HNL',
 'DCA',
 'PDX']

In [26]:
len(top_busiest)

30

In [27]:
# Convert smaller airports to OTHER category
flights_top = flights.copy().merge(meta, left_on="ORIGIN", right_on = "airport_code")


flights_top['ORIGIN'] = flights_top['ORIGIN'].apply(lambda x: x if x in top_busiest else 'OTHER')
flights_top['DEST'] = flights_top['DEST'].apply(lambda x: x if x in top_busiest else 'OTHER')



In [29]:

flights_top['timezone'] = flights_top.apply(lambda row: row['timezone'] if row['ORIGIN'] in 
                                            top_busiest else 'OTHER_TIMEZONE', axis = 1)

flights_top["airport_full_name"] = flights_top.apply(lambda row: row['airport_full_name'] if row['ORIGIN'] in 
                                            top_busiest else 'Other Airports', axis = 1)

flights_top["airport_city"] = flights_top.apply(lambda row: row['airport_city'] if row['ORIGIN'] in 
                                            top_busiest else 'Other Cities', axis = 1)

In [30]:
# flights_top = flights[(flights.ORIGIN.isin(top_busiest)) & (flights.DEST.isin(top_busiest))]

In [31]:
flights_top.shape

(9778, 39)

In [32]:
#  fl_top_merged = flights_top.merge(meta, left_on="ORIGIN", right_on = "airport_code")

In [33]:
# fl_top_merged.shape

In [34]:
# fl_top_merged.columns

In [35]:
fl_top_merged = flights_top.copy()

In [36]:
fl_top_merged['ORIGIN'].unique()

array(['CLT', 'LAS', 'SEA', 'DEN', 'SLC', 'PDX', 'SFO', 'MSP', 'LAX',
       'PHX', 'OTHER', 'HNL', 'EWR', 'BOS', 'PHL', 'ORD', 'FLL', 'DFW',
       'JFK', 'IAH', 'TPA', 'BWI', 'MCO', 'DTW', 'DCA', 'LGA', 'MIA',
       'ATL', 'DAL', 'MDW', 'SAN'], dtype=object)

In [37]:
fl_top_merged['timezone'].unique()

array(['America/New_York', 'America/Los_Angeles', 'America/Denver',
       'America/Chicago', 'America/Phoenix', 'OTHER_TIMEZONE',
       'Pacific/Honolulu'], dtype=object)

In [39]:
fl_top_merged.columns

Index([u'CARRIER_CODE', u'ORIGIN', u'ORIGIN_STATE_ABR', u'DEST',
       u'DEST_STATE_ABR', u'CRS_DEP_TIME', u'DEP_TIME', u'DEP_DELAY',
       u'CRS_ARR_TIME', u'ARR_TIME', u'ARR_DELAY', u'CANCELLED',
       u'CANCELLATION_CODE', u'ACTUAL_ELAPSED_TIME', u'DISTANCE',
       u'CARRIER_DELAY', u'WEATHER_DELAY', u'NAS_DELAY', u'SECURITY_DELAY',
       u'LATE_AIRCRAFT_DELAY', u'CARRIER_NAME', u'OriginLat', u'OriginLon',
       u'DestLat', u'DestLon', u'row_num', u'airport_full_name',
       u'airport_city', u'country', u'airport_code', u'airport_random_id',
       u'lat', u'lon', u'r_1', u'r_2', u'r_3', u'timezone', u'r_4', u'r_5'],
      dtype='object')

In [40]:
gb = fl_top_merged.groupby(['ORIGIN', 'DEST', 'timezone', 'airport_full_name',  'airport_city'], as_index=False).size()
gb.name = 'size'
gb = gb.reset_index()

In [41]:
# I guess some airports don't have any flights to other airports (30*29) = 870, but we have 742.
gb.shape


(803, 6)

In [42]:
gb.head()

,ORIGIN,DEST,timezone,airport_full_name,airport_city,size
0,ATL,BOS,America/New_York,Hartsfield Jackson Atlanta International Airport,Atlanta,10
1,ATL,BWI,America/New_York,Hartsfield Jackson Atlanta International Airport,Atlanta,11
2,ATL,CLT,America/New_York,Hartsfield Jackson Atlanta International Airport,Atlanta,10
3,ATL,DAL,America/New_York,Hartsfield Jackson Atlanta International Airport,Atlanta,3
4,ATL,DCA,America/New_York,Hartsfield Jackson Atlanta International Airport,Atlanta,10


In [43]:
gb['ORIGIN'].unique()

array(['ATL', 'BOS', 'BWI', 'CLT', 'DAL', 'DCA', 'DEN', 'DFW', 'DTW',
       'EWR', 'FLL', 'HNL', 'IAH', 'JFK', 'LAS', 'LAX', 'LGA', 'MCO',
       'MDW', 'MIA', 'MSP', 'ORD', 'OTHER', 'PDX', 'PHL', 'PHX', 'SAN',
       'SEA', 'SFO', 'SLC', 'TPA'], dtype=object)

In [44]:
set(gb['ORIGIN'].unique()) == set(top_busiest)

False

In [45]:
# Phoenix and Los Angeles follow the same time, so I make it the same color and group

tz_colgrp_map = {
    'America/Puerto_Rico' : ("#FF7F00", 1),
    'America/New_York' : ("#FF7F00", 1),
    'America/Chicago' : ("#377EB8", 2),
    'America/Denver' : ("#F781BF", 3),
    'America/Los_Angeles' : ("#A0522D", 4),
    'America/Phoenix' : ("#A0522D", 4),
    'Pacific/Honolulu' : ("#984EA3", 5),
    'OTHER_TIMEZONE' : ("#4DAF4A", 6) 
}


def get_color(timezone):    
    return tz_colgrp_map[timezone][0]

def get_group(timezone):    
    return tz_colgrp_map[timezone][1]

In [46]:
gb['color'] = gb['timezone'].apply(lambda x: get_color(x))

In [47]:
gb['group'] = gb['timezone'].apply(lambda x: get_group(x))

In [49]:
gb.shape

(803, 8)

In [50]:
flights_airports = gb.groupby(['ORIGIN', 'airport_full_name',  'airport_city', 'group', 'color'], as_index=False).sum().sort_values(['group', 'size'], ascending = [True, False]).reset_index()

In [51]:
flights_airports.sort_values(['group', 'size'], ascending = [True, False], inplace=True)

In [55]:
flights_airports['name'] = flights_airports['airport_city']

In [56]:
flights_airports.shape

(31, 8)

In [57]:
flights_airports.head()

,index,ORIGIN,airport_full_name,airport_city,group,color,size,name
0,0,ATL,Hartsfield Jackson Atlanta International Airport,Atlanta,1,#FF7F00,568,Atlanta
1,17,MCO,Orlando International Airport,Orlando,1,#FF7F00,343,Orlando
2,13,JFK,John F Kennedy International Airport,New York,1,#FF7F00,255,New York
3,10,FLL,Fort Lauderdale Hollywood International Airport,Fort Lauderdale,1,#FF7F00,248,Fort Lauderdale
4,3,CLT,Charlotte Douglas International Airport,Charlotte,1,#FF7F00,236,Charlotte


In [58]:
flights_airports.to_csv('flights-airports.csv', index=False)

In [59]:
total_flights = np.sum(gb['size'])

In [62]:
airports = list(flights_airports.ORIGIN)

In [63]:
airports

['ATL',
 'MCO',
 'JFK',
 'FLL',
 'CLT',
 'BOS',
 'BWI',
 'MIA',
 'EWR',
 'LGA',
 'TPA',
 'DTW',
 'PHL',
 'DCA',
 'DFW',
 'ORD',
 'MDW',
 'MSP',
 'IAH',
 'DAL',
 'DEN',
 'SLC',
 'LAX',
 'PHX',
 'LAS',
 'SFO',
 'SEA',
 'SAN',
 'PDX',
 'HNL',
 'OTHER']

In [64]:
adj_matrix = []
for airport_orig in airports:
    row = []
    for airport_dest in airports:
        record = gb.loc[(gb.ORIGIN == airport_orig) & (gb.DEST == airport_dest), 'size']
        if len(record) != 0:
            row.append(record.values[0]*1.0/total_flights)
        else:
            row.append(0)
    adj_matrix.append(row)

In [66]:
adj_matrix

[[0,
  0.0018408672530169769,
  0.00061362241767232563,
  0.0019431376559623645,
  0.0010227040294538761,
  0.0010227040294538761,
  0.0011249744323992637,
  0.0013295152382900388,
  0.0007158928206177132,
  0.0017385968500715893,
  0.0015340560441808142,
  0.0012272448353446513,
  0.0012272448353446513,
  0.0010227040294538761,
  0.0013295152382900388,
  0.00092043362650848844,
  0.0007158928206177132,
  0.0011249744323992637,
  0.00081816322356310087,
  0.00030681120883616281,
  0.0014317856412354264,
  0.0007158928206177132,
  0.0014317856412354264,
  0.00092043362650848844,
  0.0010227040294538761,
  0.00081816322356310087,
  0.00081816322356310087,
  0.00051135201472693806,
  0.00030681120883616281,
  0.00010227040294538761,
  0.028431172018817754],
 [0.0017385968500715893,
  0,
  0.0017385968500715893,
  0.00020454080589077522,
  0.00092043362650848844,
  0.0010227040294538761,
  0.0011249744323992637,
  0.00092043362650848844,
  0.0019431376559623645,
  0.0012272448353446513,
  

In [67]:
text_file = open("flights-matrix.json", "w")
text_file.write(str(adj_matrix))
text_file.close()